# Virgo: exploratory data analysis

starting

## Libraries

In [1]:
import yfinance as yf
import pandas as pd
import numpy as np

In [2]:
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import seaborn as sns; sns.set()

import plotly.express as px
import plotly.offline as pyo

In [3]:
import datetime
from dateutil.relativedelta import relativedelta

In [4]:
import data_functions
import configs

## Transfomations

In [5]:
n_days = configs.data_configs.n_days
lag_days = configs.data_configs.lags
window = configs.data_configs.window
ref_price = configs.data_configs.ref_price
std_column = configs.data_configs.std_column
logdif_column = configs.data_configs.logdif_column

In [11]:
print(f'the return is given for {lag_days} days')

the return is given for 3 days


In [6]:
appl_stock = data_functions.get_stock_data(stock_code = "AAPL", n_days = n_days, window = window, lags = lag_days)
appl_stock = data_functions.shape_data(appl_stock, 'apple', ref_price, std_column, logdif_column)

In [10]:
test_data = '2022-12-01'### higher equal

In [12]:
appl_stock

,Date,apple_price,apple_stv,apple_logdif
10370,2022-01-28,169.347137,3.897075,0.063940
10371,2022-01-31,173.771454,6.070753,0.090294
10372,2022-02-01,173.602463,7.181807,0.092268
10373,2022-02-02,174.825333,7.805710,0.031837
10374,2022-02-03,171.902313,7.126177,-0.010815
...,...,...,...,...
10607,2023-01-09,130.149994,2.370321,0.029553
10608,2023-01-10,130.729996,2.532091,0.044660
10609,2023-01-11,133.490005,3.219177,0.029420
10610,2023-01-12,133.410004,3.223493,0.024740
